# Here Ill try to convert images to TF data

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.5.0


In [3]:
work_file_dir = os.path.dirname(os.path.realpath('__file__'))
data_dir = pathlib.Path(work_file_dir+'/data')
my_class1 = list(data_dir.glob('data/train/bill/*'))
my_class2 = list(data_dir.glob('data/train/upd/*'))
test_class = list(data_dir.glob(f'{data_dir}/test/*'))
data_dir

PosixPath('/Users/johnlennon/My Drive/Python/Doc_mentor/Doc_mentor/data')

In [ ]:
PIL.Image.open(str(my_class1[0]))

In [ ]:
# load image via tf.io
img = tf.io.read_file(str(my_class1[0]))
# img.set_shape([28, 28, 3])
# convert to tensor (specify 3 channels explicitly since png files contains additional alpha channel)
# set the dtypes to align with pytorch for comparison since it will use uint8 by default
tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
# (384, 470, 3)
# resize tensor to 224 x 224
tensor = tf.image.resize(tensor, [224, 224])
# (224, 224, 3)
# add another dimension at the front to get NHWC shape
input_tensor = tf.expand_dims(tensor, axis=0)
# (1, 224, 224, 3)
# transpose (224, 224, 3) -> (3, 224, 224)
tensor = tf.transpose(tensor, perm=[2, 0, 1])

In [ ]:
tf.shape(tensor)

In [ ]:
tf.transpose(tensor).shape

In [ ]:
tensor.shape

In [ ]:
str(my_class1[1])

In [ ]:
plt.imshow(tf.transpose(tensor), cmap="Greys")

In [ ]:
batch_size = 12
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.flow_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Train with "flow_from_directory"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = '/content/drive/MyDrive/Python/Doc_mentor/Doc_mentor/data'

In [4]:
from tensorflow.keras.backend import clear_session
clear_session()

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(150,150,3,), name="layer1")) # первый скрытый слой
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128, activation='relu', name="layer2")) # второй скрытый слой
model.add(Dense(3, activation='softmax', name="output")) # выходной слой

2022-02-18 15:48:54.075292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-18 15:48:54.075543: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [6]:
model.compile(
    loss='sparse_categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',  # так будет быстрее, позже узнаем что это :)
    metrics=['accuracy']  # выводим процент правильных ответов
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 150, 150, 256)     1024      
_________________________________________________________________
flatten (Flatten)            (None, 5760000)           0         
_________________________________________________________________
layer2 (Dense)               (None, 128)               737280128 
_________________________________________________________________
output (Dense)               (None, 3)                 387       
Total params: 737,281,539
Trainable params: 737,281,539
Non-trainable params: 0
_________________________________________________________________


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        f'{data_dir}/train',
        target_size=(150, 150),
        batch_size=5,
        class_mode='binary')
validation_generator = val_datagen.flow_from_directory(
        f'{data_dir}/val',
        target_size=(150, 150),
        batch_size=5,
        class_mode='binary')


Found 567 images belonging to 3 classes.
Found 114 images belonging to 3 classes.


In [8]:
model.fit(train_generator,
        steps_per_epoch=20,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=5)

2022-02-18 15:49:00.755851: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-18 15:49:00.756003: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-18 15:49:00.863569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/30
20/20 [==============================] - ETA: 0s - loss: 139.7224 - accuracy: 0.4200

2022-02-18 15:50:25.896623: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 88s 4s/step - loss: 139.7224 - accuracy: 0.4200 - val_loss: 1.1003 - val_accuracy: 0.1600
Epoch 2/30
20/20 [==============================] - 83s 4s/step - loss: 3.0677 - accuracy: 0.3600 - val_loss: 1.0990 - val_accuracy: 0.4000
Epoch 3/30
20/20 [==============================] - 86s 4s/step - loss: 1.0984 - accuracy: 0.2800 - val_loss: 1.1025 - val_accuracy: 0.2000
Epoch 4/30
20/20 [==============================] - 82s 4s/step - loss: 2.5015 - accuracy: 0.3300 - val_loss: 1.1009 - val_accuracy: 0.3200
Epoch 5/30
20/20 [==============================] - 72s 4s/step - loss: 1.5606 - accuracy: 0.4500 - val_loss: 1.1025 - val_accuracy: 0.2400
Epoch 6/30
20/20 [==============================] - 52s 2s/step - loss: 1.0968 - accuracy: 0.4100 - val_loss: 1.1100 - val_accuracy: 0.0800
Epoch 7/30
20/20 [==============================] - 69s 3s/step - loss: 5.0341 - accuracy: 0.3400 - val_loss: 1.1050 - val_accuracy: 0.3600
Epoch 8/30
20/20 [===========

In [ ]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [10]:
# Save model v1
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

model.save('data/saved_models/my_model2')

2.5.0


2022-02-18 16:28:49.716898: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: data/saved_models/my_model2/assets


In [9]:
# Save model v2
# always save your weights after training or during training
model.save_weights('first_try.h5')  

In [15]:
# load model
new_model = tf.keras.models.load_model(f'{data_dir}/saved_models/my_model1')

In [ ]:
X_train = []
y_train = []
for x,y in train_generator:
    print(x.shape)
    print(x[0].shape, y.shape)
    print(x[1].shape, y.shape)
    print(y)
    print(len(x))
    
    break


In [ ]:
tf.config.list_physical_devices()

In [ ]:
my_np_array1 = np.array([[[1],[2],[3]],[[4],[5],[6]]], dtype=object)
my_np_array2 = np.array([[[1],[2],[3]],[[4],[5],[6]]], dtype=object)
my_np_array = my_np_array1 + my_np_array2
np.shape(my_np_array)


In [ ]:
np.add(my_np_array1, my_np_array2)

In [ ]:
my_np_array1.__add__(my_np_array2)

In [ ]:
my_np_array = np.concatenate((my_np_array1, my_np_array2))
np.shape(my_np_array)

In [ ]:
a = [[[1],[2],[3]],[[4],[5],[6]]]

b = np.array([a]+[a])
np.shape(b)

In [ ]:
np.shape([a])

### Testing model

#### Test variant1

In [70]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        f'{data_dir}/test',
        target_size=(150, 150),
        batch_size=5,
        class_mode='binary')

Found 24 images belonging to 1 classes.


In [23]:
test_class = list(data_dir.glob('test/*.jpg'))
len(test_class)
test_class[0]

PosixPath('/Users/johnlennon/My Drive/Python/Doc_mentor/Doc_mentor/data/test/202103_test_18.jpg')

In [58]:
test_class = list(data_dir.glob('test/*.jpg'))
img = tf.io.read_file(str(test_class[0]))
# img.set_shape([28, 28, 3])
# convert to tensor (specify 3 channels explicitly since png files contains additional alpha channel)
# set the dtypes to align with pytorch for comparison since it will use uint8 by default
tensor = tf.io.decode_image(img, channels=0, dtype=tf.dtypes.float32)
# (384, 470, 3)
# resize tensor to 224 x 224
tensor = tf.image.resize(tensor, [150, 150])
# (224, 224, 3)
# add another dimension at the front to get NHWC shape
input_tensor = tf.expand_dims(tensor, axis=0)
# (1, 224, 224, 3)
# transpose (224, 224, 3) -> (3, 224, 224)
# tensor = tf.transpose(tensor, perm=[2, 0, 1])


##### tensor.shape

In [71]:

model.predict(test_generator)
#plt.imshow(tf.transpose(tensor), cmap="Greys")

array([[0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.37316987, 0.3241894 , 0.30264077],
       [0.

In [ ]:
# Тут ваш код Архитектуры сети
# не забудьте 10 выходов с активацией softmax на выходном слое!
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(28*28,))) # первый скрытый слой
model.add(Dense(128, activation='relu',)) # второй скрытый слой
model.add(Dense(10, activation='softmax',)) # выходной слой

In [ ]:
import tensorflow as tf
